In [1]:
from pylsl import StreamInlet, resolve_stream
import numpy as np
import joblib  # Used for loading sklearn models
from preprocess import preprocess_data #modify path
# from model import YourModel  # This might be where you define any custom transformers or pipelines, if needed
# from preprocessing import preprocess_data  # Assume you have a preprocessing function
# from feature_extraction import extract_features  # Assume you have a feature extraction function

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Configuration
#model_path = 'path/to/your/saved/model.joblib'  # Path to your trained model
srate = 160  # Sampling rate of the EEG data
epoch_length_sec = 5  # Length of an epoch in seconds
samples_per_epoch = srate * epoch_length_sec  # Number of samples per epoch

# Load your model
#model = joblib.load(model_path)

# Resolve the stream
print("Looking for an EEG stream...")
streams = resolve_stream('type', 'EEG', 'name', 'BioSemi')
inlet = StreamInlet(streams[0])

# Initialize a buffer for accumulating samples
buffer = []

def decode(epoch):
    """
    Process and decode an epoch of EEG data.
    """
    # Assuming you have defined preprocess_data and extract_features functions
    preprocessed_epoch = preprocess_data(epoch)  # Preprocess the epoch
    features = extract_features(preprocessed_epoch)  # Extract features
    prediction = model.predict([features])  # Sklearn expects a 2D array for the features
    return prediction

def preprocess_data(epoch):
    # Placeholder for your preprocessing steps
    return epoch  # Return the processed epoch

def extract_features(epoch):
    # Placeholder for your feature extraction logic
    return epoch.mean(axis=0) 

while True:
    # Pull sample from LSL stream
    sample, timestamp = inlet.pull_sample()
    buffer.append(sample)
    
    # Check if buffer has enough samples to form an epoch
    if len(buffer) >= samples_per_epoch:
        epoch = np.array(buffer[:samples_per_epoch])  
        buffer = buffer[samples_per_epoch:]  
        
        # Decode the epoch
        prediction = decode(epoch)
        print(f"Timestamp: {timestamp}, Prediction: {prediction}")


Looking for an EEG stream...


2024-03-10 22:29:00.998 (  18.560s) [          42BA5D]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-10 22:29:00.999 (  18.560s) [          42BA5D]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-10 22:29:00.999 (  18.560s) [          42BA5D]      netinterfaces.cpp:102   INFO| 	IPv4 addr: 7f000001
2024-03-10 22:29:00.999 (  18.560s) [          42BA5D]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-10 22:29:00.999 (  18.560s) [          42BA5D]      netinterfaces.cpp:105   INFO| 	IPv6 addr: ::1
2024-03-10 22:29:00.999 (  18.560s) [          42BA5D]      netinterfaces.cpp:91    INFO| netif 'lo0' (status: 1, multicast: 32768, broadcast: 0)
2024-03-10 22:29:00.999 (  18.560s) [          42BA5D]      netinterfaces.cpp:105   INFO| 	IPv6 addr: fe80::1%lo0
2024-03-10 22:29:00.999 (  18.560s) [          42BA5D]      netinterfaces.cpp:91    I